In [1]:
!pip install keras==1.2.2

     |████████████████████████████████| 184kB 5.4MB/s 
     |████████████████████████████████| 2.8MB 31.6MB/s 
  Created wheel for keras: filename=Keras-1.2.2-cp37-none-any.whl size=209602 sha256=43a1cffabac3543c1c499edf0f75a31cff1cc7485d0e14553947e0fb63cd6dbb
  Stored in directory: /root/.cache/pip/wheels/55/07/cf/b32db0a8d243b2fd6759d5d7cb650aa20670b2b740209cbf7e
  Created wheel for theano: filename=Theano-1.0.5-cp37-none-any.whl size=2668111 sha256=4403192dfb03e855a1f1781cbc3263f1ceba9cf98bd69a3fff06178e0fd37d9c
  Stored in directory: /root/.cache/pip/wheels/89/40/74/3a0b7d937890c66c4373120117ebf4ba99f4889b4a0a6a810c
Successfully built keras theano
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [2]:
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7MB 31kB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 3.2MB 31.8MB/s 
     |████████████████████████████████| 368kB 29.2MB/s 
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.13.2 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0


In [ ]:
import glob

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

from tqdm import tqdm
import pandas as pd

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Merge, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image

import nltk

In [7]:
file_location_caption = '/content/Flickr8k.token.txt'
file_location_caption_unique = '/content/unique_captions.p'
file_location_training_data = '/content/flickr_8k_train_dataset.txt'

file_location_train_encode = '/content/encoded_images_train.p'

# Preprocessing

In [8]:
captions = open(file_location_caption, 'r').read().strip().split('\n')
unique = pickle.load(open(file_location_caption_unique, "rb"))

train_data_encode = pickle.load(open(file_location_train_encode, 'rb'))

In [9]:
word2idx = {val:index for index, val in enumerate(unique)}
idx2word = {index:val for index, val in enumerate(unique)}

In [10]:
max_cap_length = 40
vocab_size = 8256

In [11]:
dictionary_captions = {}
for i, row in enumerate(captions):
    row = row.split('\t')
    row[0] = row[0][:len(row[0])-2]
    if row[0] in dictionary_captions:
        dictionary_captions[row[0]].append(row[1])
    else:
        dictionary_captions[row[0]] = [row[1]]

In [12]:
def data_generator(batch_size= 64):
    partial_captions = []
    next_word = []
    images= []

    df = pd.read_csv(file_location_training_data, delimiter='\t')
    df = df.sample(frac=1)
    iter = df.iterrows()

    c = []
    imgs = []

    for i in range(df.shape[0]):
        x = next(iter)
        c.append(x[1][1])
        imgs.append(x[1][0])

    count = 0
    while True:
        for idx, text in enumerate(c):
            current_image = train_data_encode[imgs[idx]]

            for i in range(len(text.split()) - 1):
                count += 1

                partial = [word2idx[txt] for txt in text.split()[:i+1]]
                partial_captions.append(partial)

                n = np.zeros(vocab_size)
                n[word2idx[text.split()[i+1]]] = 1
                next_word.append(n)

                images.append(current_image)

                if count >= batch_size:
                    next_word = np.asarray(next_word)
                    images = np.asarray(images)
                    partial_captions = sequence.pad_sequences(partial_captions, maxlen=max_cap_length, padding='post')
                    yield[[images, partial_captions], next_word]

                    partial_captions = []
                    next_word = []
                    images = []
                    count = 0

In [13]:
embedding_size = 300

image_model = Sequential([
        Dense(embedding_size, input_shape=(2048,), activation='relu'),
        RepeatVector(max_cap_length)
    ])

caption_model = Sequential([
        Embedding(vocab_size, embedding_size, input_length=max_cap_length),
        LSTM(256, return_sequences=True),
        TimeDistributed(Dense(300))
    ])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
model = Sequential([
        Merge([image_model, caption_model], mode='concat', concat_axis=1),
        Bidirectional(LSTM(256, return_sequences=False)),
        Dense(vocab_size),
        Activation('softmax')
    ])

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics= ['accuracy'])

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 300)           614700                                       
____________________________________________________________________________________________________
repeatvector_1 (RepeatVector)    (None, 40, 300)       0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 40, 300)       2476800                                      
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 40, 256)       570368                                       
___________________________________________________________________________________________

In [19]:
model.fit_generator(data_generator(batch_size=512), samples_per_epoch=25545, nb_epoch=1, verbose=1)

Epoch 1/1
25600/25545 [==============================] - 870s - loss: 5.3205 - acc: 0.1573    


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


In [20]:
model.fit_generator(data_generator(batch_size=512), samples_per_epoch=35545, nb_epoch=1, verbose=1)

Epoch 1/1
35840/35545 [==============================] - 1239s - loss: 5.1103 - acc: 0.1927  


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


In [21]:
model.fit_generator(data_generator(batch_size=1024), samples_per_epoch=45545, nb_epoch=1, verbose=1)

Epoch 1/1
46080/45545 [==============================] - 1577s - loss: 4.9317 - acc: 0.2271   


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


In [ ]:
model.fit_generator(data_generator(batch_size=1024), samples_per_epoch=55545, nb_epoch=1, verbose=1)

Epoch 1/1
52224/55545 [===========================>..] - ETA: 113s - loss: 4.8086 - acc: 0.2424